## **Round Robin (RR) Code**

In [4]:
from collections import deque

sample_processes=[
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results):
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}")
    for r in results:
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def round_robin(sample_processes, quantum):
    queue = deque()
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes]
    time = 0
    results = {}

    sample_processes = sorted(sample_processes, key=lambda x: x["arrival"])
    
    i = 0
    while i < len(sample_processes) or queue:
        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time:
            queue.append(sample_processes[i])
            i += 1

        if not queue:
            time = sample_processes[i]["arrival"]
            continue

        p = queue.popleft()

        if p["pid"] not in results:
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        run_time = min(quantum, p["remaining"])
        p["remaining"] -= run_time
        time += run_time

        while i < len(sample_processes) and sample_processes[i]["arrival"] <= time:
            queue.append(sample_processes[i])
            i += 1

        if p["remaining"] > 0:
            queue.append(p)
        else:
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"])
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"])

    return list(results.values())

print("Round Robin (q=3):")
print_schedule(round_robin(sample_processes, quantum=3))

Round Robin (q=3):
PID   Start  Finish Waiting Turnaround
P1    0      32     22      32        
P2    3      18     11      17        
P3    6      34     21      32        
P4    9      31     21      28        


## **Gantt Chart & Averages Code**

In [6]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"])

    chart = ""
    timeline = ""

    for r in results:
        length = r["finish"] - r["start"]
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}"

    timeline += f"{results[-1]['finish']}"

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results:
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-")
    print(label_line + "|")

    print(timeline)

def compute_averages(results):
    total_wait = sum(r["waiting"] for r in results)
    total_turn = sum(r["turnaround"] for r in results)
    n = len(results)
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n}

results = round_robin(sample_processes, quantum=3)
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])


Gantt Chart:
|--------------------------------|---------------|----------------------------|----------------------|
|---------------P1---------------|-------P2------|-------------P3-------------|----------P4----------|
0                                3               6                            9                      31

Average Waiting Time = 18.75
Average Turnaround Time = 27.25
